# 05. 딥러닝 (Deep Learning - DNN, CNN, RNN)

---

## 목차 / Table of Contents
1. PyTorch 기본 설정 / PyTorch Setup
2. DNN (Deep Neural Network) - 정형 데이터
3. CNN (Convolutional Neural Network) - 이미지/시퀀스
4. RNN/LSTM - 시계열 데이터
5. 모델 학습 및 평가 / Training & Evaluation

In [ ]:
# ============================================================
# 라이브러리 임포트 / Library Imports
# ============================================================

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

# PyTorch
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, TensorDataset

# 사이킷런 / Scikit-learn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, accuracy_score, f1_score, classification_report

# 설정 / Settings
RANDOM_STATE = 42
np.random.seed(RANDOM_STATE)
torch.manual_seed(RANDOM_STATE)

# 디바이스 설정 / Device setup
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"✅ PyTorch version: {torch.__version__}")
print(f"✅ Device: {device}")

---
## 1. PyTorch 기본 설정 / PyTorch Setup

In [ ]:
# ============================================================
# 데이터 생성 함수 / Data Generation Functions
# ============================================================

def create_tabular_data(n_samples=1000, n_features=20, task='regression', random_state=42):
    """
    정형 데이터 생성 (DNN용)
    Create tabular data for DNN
    """
    np.random.seed(random_state)
    X = np.random.randn(n_samples, n_features)
    
    if task == 'regression':
        y = X[:, :5].sum(axis=1) + 0.5 * X[:, 0]**2 + np.random.randn(n_samples) * 0.1
    else:  # classification
        y = (X[:, :3].sum(axis=1) > 0).astype(int)
    
    return X.astype(np.float32), y.astype(np.float32 if task == 'regression' else np.int64)

def create_sequence_data(n_samples=500, seq_length=50, n_features=1, random_state=42):
    """
    시퀀스 데이터 생성 (RNN용)
    Create sequence data for RNN
    """
    np.random.seed(random_state)
    
    X = []
    y = []
    
    for _ in range(n_samples):
        # 사인파 + 노이즈 / Sine wave + noise
        t = np.linspace(0, 4*np.pi, seq_length + 1)
        signal = np.sin(t) + np.random.randn(seq_length + 1) * 0.1
        
        X.append(signal[:-1].reshape(-1, n_features))
        y.append(signal[-1])  # 다음 값 예측 / Predict next value
    
    return np.array(X, dtype=np.float32), np.array(y, dtype=np.float32)

print("✅ 데이터 생성 함수 정의 완료 / Data generation functions defined")

In [ ]:
# ============================================================
# 학습 유틸리티 함수 / Training Utility Functions
# ============================================================

def train_model(model, train_loader, criterion, optimizer, epochs=50, val_loader=None, verbose=True):
    """
    모델 학습 함수
    Model training function
    """
    model.to(device)
    train_losses = []
    val_losses = []
    
    for epoch in range(epochs):
        # Training
        model.train()
        train_loss = 0
        
        for X_batch, y_batch in train_loader:
            X_batch, y_batch = X_batch.to(device), y_batch.to(device)
            
            optimizer.zero_grad()
            outputs = model(X_batch)
            loss = criterion(outputs.squeeze(), y_batch)
            loss.backward()
            optimizer.step()
            
            train_loss += loss.item()
        
        train_loss /= len(train_loader)
        train_losses.append(train_loss)
        
        # Validation
        if val_loader:
            model.eval()
            val_loss = 0
            with torch.no_grad():
                for X_batch, y_batch in val_loader:
                    X_batch, y_batch = X_batch.to(device), y_batch.to(device)
                    outputs = model(X_batch)
                    loss = criterion(outputs.squeeze(), y_batch)
                    val_loss += loss.item()
            val_loss /= len(val_loader)
            val_losses.append(val_loss)
        
        if verbose and (epoch + 1) % 10 == 0:
            if val_loader:
                print(f"Epoch [{epoch+1}/{epochs}] - Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}")
            else:
                print(f"Epoch [{epoch+1}/{epochs}] - Train Loss: {train_loss:.4f}")
    
    return train_losses, val_losses

def evaluate_regression(model, test_loader):
    """
    회귀 모델 평가
    Evaluate regression model
    """
    model.eval()
    predictions = []
    actuals = []
    
    with torch.no_grad():
        for X_batch, y_batch in test_loader:
            X_batch = X_batch.to(device)
            outputs = model(X_batch)
            predictions.extend(outputs.squeeze().cpu().numpy())
            actuals.extend(y_batch.numpy())
    
    predictions = np.array(predictions)
    actuals = np.array(actuals)
    
    rmse = np.sqrt(mean_squared_error(actuals, predictions))
    print(f"RMSE: {rmse:.4f}")
    
    return predictions, actuals

def evaluate_classification(model, test_loader):
    """
    분류 모델 평가
    Evaluate classification model
    """
    model.eval()
    predictions = []
    actuals = []
    
    with torch.no_grad():
        for X_batch, y_batch in test_loader:
            X_batch = X_batch.to(device)
            outputs = model(X_batch)
            _, predicted = torch.max(outputs, 1)
            predictions.extend(predicted.cpu().numpy())
            actuals.extend(y_batch.numpy())
    
    accuracy = accuracy_score(actuals, predictions)
    f1 = f1_score(actuals, predictions, average='weighted')
    print(f"Accuracy: {accuracy:.4f}")
    print(f"F1 Score: {f1:.4f}")
    
    return predictions, actuals

print("✅ 학습 유틸리티 함수 정의 완료 / Training utilities defined")

---
## 2. DNN (Deep Neural Network) - 정형 데이터

**적합한 데이터**: 테이블 형태의 정형 데이터
**Suitable for**: Tabular/structured data

In [ ]:
# ============================================================
# DNN 모델 정의 (회귀용) / DNN Model Definition (Regression)
# ============================================================

class DNNRegressor(nn.Module):
    """
    DNN 회귀 모델
    DNN Regression Model
    
    구조: Input -> Hidden1 -> Hidden2 -> Hidden3 -> Output
    """
    def __init__(self, input_dim, hidden_dims=[128, 64, 32], dropout=0.2):
        super(DNNRegressor, self).__init__()
        
        layers = []
        prev_dim = input_dim
        
        for hidden_dim in hidden_dims:
            layers.append(nn.Linear(prev_dim, hidden_dim))
            layers.append(nn.BatchNorm1d(hidden_dim))  # 배치 정규화 / Batch normalization
            layers.append(nn.ReLU())
            layers.append(nn.Dropout(dropout))
            prev_dim = hidden_dim
        
        layers.append(nn.Linear(prev_dim, 1))  # 출력층 / Output layer
        
        self.network = nn.Sequential(*layers)
    
    def forward(self, x):
        return self.network(x)

# 모델 확인 / Check model
sample_model = DNNRegressor(input_dim=20)
print(sample_model)

In [ ]:
# ============================================================
# DNN 회귀 학습 / DNN Regression Training
# ============================================================

# 데이터 생성 / Generate data
X, y = create_tabular_data(n_samples=1000, n_features=20, task='regression')

# 분할 / Split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=RANDOM_STATE)

# 스케일링 / Scaling
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train).astype(np.float32)
X_test_scaled = scaler.transform(X_test).astype(np.float32)

# DataLoader 생성 / Create DataLoader
train_dataset = TensorDataset(torch.tensor(X_train_scaled), torch.tensor(y_train))
test_dataset = TensorDataset(torch.tensor(X_test_scaled), torch.tensor(y_test))

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# 모델 생성 / Create model
dnn_reg = DNNRegressor(input_dim=20, hidden_dims=[128, 64, 32], dropout=0.2)

# 손실 함수 및 옵티마이저 / Loss and optimizer
criterion = nn.MSELoss()
optimizer = optim.Adam(dnn_reg.parameters(), lr=0.001)

# 학습 / Train
print("DNN 회귀 모델 학습 시작...")
train_losses, _ = train_model(dnn_reg, train_loader, criterion, optimizer, epochs=50)

In [ ]:
# ============================================================
# DNN 회귀 평가 / DNN Regression Evaluation
# ============================================================

print("\n📊 DNN 회귀 평가 결과:")
predictions, actuals = evaluate_regression(dnn_reg, test_loader)

# 학습 곡선 시각화 / Visualize learning curve
fig, axes = plt.subplots(1, 2, figsize=(12, 4))

axes[0].plot(train_losses)
axes[0].set_xlabel('Epoch')
axes[0].set_ylabel('Loss')
axes[0].set_title('Training Loss')

axes[1].scatter(actuals, predictions, alpha=0.5)
axes[1].plot([min(actuals), max(actuals)], [min(actuals), max(actuals)], 'r--')
axes[1].set_xlabel('Actual')
axes[1].set_ylabel('Predicted')
axes[1].set_title('Actual vs Predicted')

plt.tight_layout()
plt.show()

In [ ]:
# ============================================================
# DNN 분류 모델 / DNN Classification Model
# ============================================================

class DNNClassifier(nn.Module):
    """
    DNN 분류 모델
    DNN Classification Model
    """
    def __init__(self, input_dim, hidden_dims=[128, 64, 32], num_classes=2, dropout=0.2):
        super(DNNClassifier, self).__init__()
        
        layers = []
        prev_dim = input_dim
        
        for hidden_dim in hidden_dims:
            layers.append(nn.Linear(prev_dim, hidden_dim))
            layers.append(nn.BatchNorm1d(hidden_dim))
            layers.append(nn.ReLU())
            layers.append(nn.Dropout(dropout))
            prev_dim = hidden_dim
        
        layers.append(nn.Linear(prev_dim, num_classes))
        
        self.network = nn.Sequential(*layers)
    
    def forward(self, x):
        return self.network(x)

# 분류 데이터 생성 / Generate classification data
X_clf, y_clf = create_tabular_data(n_samples=1000, n_features=20, task='classification')
X_train_clf, X_test_clf, y_train_clf, y_test_clf = train_test_split(
    X_clf, y_clf, test_size=0.2, random_state=RANDOM_STATE, stratify=y_clf
)

# 스케일링 / Scaling
X_train_clf_scaled = scaler.fit_transform(X_train_clf).astype(np.float32)
X_test_clf_scaled = scaler.transform(X_test_clf).astype(np.float32)

# DataLoader
train_clf_dataset = TensorDataset(torch.tensor(X_train_clf_scaled), torch.tensor(y_train_clf))
test_clf_dataset = TensorDataset(torch.tensor(X_test_clf_scaled), torch.tensor(y_test_clf))

train_clf_loader = DataLoader(train_clf_dataset, batch_size=32, shuffle=True)
test_clf_loader = DataLoader(test_clf_dataset, batch_size=32, shuffle=False)

# 모델 생성 및 학습 / Create and train model
dnn_clf = DNNClassifier(input_dim=20, num_classes=2)
criterion_clf = nn.CrossEntropyLoss()
optimizer_clf = optim.Adam(dnn_clf.parameters(), lr=0.001)

print("DNN 분류 모델 학습 시작...")
train_model(dnn_clf, train_clf_loader, criterion_clf, optimizer_clf, epochs=50)

print("\n📊 DNN 분류 평가 결과:")
evaluate_classification(dnn_clf, test_clf_loader)

---
## 3. CNN (Convolutional Neural Network)

**적합한 데이터**: 이미지, 1D 시퀀스 데이터
**Suitable for**: Images, 1D sequence data

In [ ]:
# ============================================================
# 1D CNN 모델 (시퀀스/시계열용) / 1D CNN Model (for sequences/time series)
# ============================================================

class CNN1D(nn.Module):
    """
    1D CNN 모델 (시계열/시퀀스 데이터용)
    1D CNN Model for time series/sequence data
    
    입력: (batch_size, seq_length, n_features)
    Input: (batch_size, seq_length, n_features)
    """
    def __init__(self, input_channels=1, seq_length=50, num_classes=1):
        super(CNN1D, self).__init__()
        
        # Conv 레이어들 / Conv layers
        self.conv1 = nn.Conv1d(input_channels, 32, kernel_size=3, padding=1)
        self.conv2 = nn.Conv1d(32, 64, kernel_size=3, padding=1)
        self.conv3 = nn.Conv1d(64, 128, kernel_size=3, padding=1)
        
        self.pool = nn.MaxPool1d(kernel_size=2)
        self.dropout = nn.Dropout(0.25)
        
        # 풀링 후 시퀀스 길이 계산 / Calculate sequence length after pooling
        # 3번의 풀링: seq_length -> seq_length/2 -> seq_length/4 -> seq_length/8
        final_seq_len = seq_length // 8
        
        # 완전 연결층 / Fully connected layers
        self.fc1 = nn.Linear(128 * final_seq_len, 64)
        self.fc2 = nn.Linear(64, num_classes)
    
    def forward(self, x):
        # x: (batch, seq_len, features) -> (batch, features, seq_len)
        x = x.permute(0, 2, 1)
        
        # Conv blocks
        x = self.pool(torch.relu(self.conv1(x)))
        x = self.pool(torch.relu(self.conv2(x)))
        x = self.pool(torch.relu(self.conv3(x)))
        x = self.dropout(x)
        
        # Flatten
        x = x.view(x.size(0), -1)
        
        # FC layers
        x = torch.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)
        
        return x

print("CNN1D 모델 구조:")
cnn_sample = CNN1D(input_channels=1, seq_length=48)  # 48 = 6*8 (8로 나누어 떨어지도록)
print(cnn_sample)

In [ ]:
# ============================================================
# CNN 학습 예시 / CNN Training Example
# ============================================================

# 시퀀스 데이터 생성 / Generate sequence data
X_seq, y_seq = create_sequence_data(n_samples=500, seq_length=48, n_features=1)

# 분할 / Split
X_train_seq, X_test_seq, y_train_seq, y_test_seq = train_test_split(
    X_seq, y_seq, test_size=0.2, random_state=RANDOM_STATE
)

# DataLoader
train_seq_dataset = TensorDataset(torch.tensor(X_train_seq), torch.tensor(y_train_seq))
test_seq_dataset = TensorDataset(torch.tensor(X_test_seq), torch.tensor(y_test_seq))

train_seq_loader = DataLoader(train_seq_dataset, batch_size=32, shuffle=True)
test_seq_loader = DataLoader(test_seq_dataset, batch_size=32, shuffle=False)

# 모델 생성 및 학습 / Create and train model
cnn_model = CNN1D(input_channels=1, seq_length=48, num_classes=1)
criterion_cnn = nn.MSELoss()
optimizer_cnn = optim.Adam(cnn_model.parameters(), lr=0.001)

print("CNN 모델 학습 시작...")
train_model(cnn_model, train_seq_loader, criterion_cnn, optimizer_cnn, epochs=50)

print("\n📊 CNN 평가 결과:")
evaluate_regression(cnn_model, test_seq_loader)

---
## 4. RNN/LSTM - 시계열 데이터

**적합한 데이터**: 시계열, 순차적 데이터
**Suitable for**: Time series, sequential data

In [ ]:
# ============================================================
# LSTM 모델 정의 / LSTM Model Definition
# ============================================================

class LSTMModel(nn.Module):
    """
    LSTM 모델 (시계열 예측용)
    LSTM Model for time series forecasting
    
    입력: (batch_size, seq_length, input_dim)
    Input: (batch_size, seq_length, input_dim)
    """
    def __init__(self, input_dim=1, hidden_dim=64, num_layers=2, output_dim=1, dropout=0.2):
        super(LSTMModel, self).__init__()
        
        self.hidden_dim = hidden_dim
        self.num_layers = num_layers
        
        # LSTM 레이어 / LSTM layers
        self.lstm = nn.LSTM(
            input_size=input_dim,
            hidden_size=hidden_dim,
            num_layers=num_layers,
            batch_first=True,  # (batch, seq, feature) 형태
            dropout=dropout if num_layers > 1 else 0
        )
        
        # 출력층 / Output layer
        self.fc = nn.Linear(hidden_dim, output_dim)
    
    def forward(self, x):
        # LSTM 출력 / LSTM output
        # lstm_out: (batch, seq_len, hidden_dim)
        lstm_out, (h_n, c_n) = self.lstm(x)
        
        # 마지막 타임스텝의 출력만 사용 / Use only last timestep output
        out = self.fc(lstm_out[:, -1, :])
        
        return out

print("LSTM 모델 구조:")
lstm_sample = LSTMModel(input_dim=1, hidden_dim=64, num_layers=2)
print(lstm_sample)

In [ ]:
# ============================================================
# LSTM 학습 / LSTM Training
# ============================================================

# 모델 생성 / Create model
lstm_model = LSTMModel(input_dim=1, hidden_dim=64, num_layers=2, output_dim=1)
criterion_lstm = nn.MSELoss()
optimizer_lstm = optim.Adam(lstm_model.parameters(), lr=0.001)

print("LSTM 모델 학습 시작...")
train_model(lstm_model, train_seq_loader, criterion_lstm, optimizer_lstm, epochs=50)

print("\n📊 LSTM 평가 결과:")
predictions_lstm, actuals_lstm = evaluate_regression(lstm_model, test_seq_loader)

In [ ]:
# ============================================================
# GRU 모델 (LSTM 대안) / GRU Model (LSTM Alternative)
# ============================================================

class GRUModel(nn.Module):
    """
    GRU 모델
    - LSTM보다 파라미터가 적어 빠름
    - 비슷한 성능
    """
    def __init__(self, input_dim=1, hidden_dim=64, num_layers=2, output_dim=1, dropout=0.2):
        super(GRUModel, self).__init__()
        
        self.gru = nn.GRU(
            input_size=input_dim,
            hidden_size=hidden_dim,
            num_layers=num_layers,
            batch_first=True,
            dropout=dropout if num_layers > 1 else 0
        )
        
        self.fc = nn.Linear(hidden_dim, output_dim)
    
    def forward(self, x):
        gru_out, h_n = self.gru(x)
        out = self.fc(gru_out[:, -1, :])
        return out

# GRU 학습 / GRU Training
gru_model = GRUModel(input_dim=1, hidden_dim=64, num_layers=2)
optimizer_gru = optim.Adam(gru_model.parameters(), lr=0.001)

print("GRU 모델 학습 시작...")
train_model(gru_model, train_seq_loader, criterion_lstm, optimizer_gru, epochs=50)

print("\n📊 GRU 평가 결과:")
evaluate_regression(gru_model, test_seq_loader)

---
## 5. 모델 저장 및 로드 / Model Save & Load

In [ ]:
# ============================================================
# 모델 저장 / Save Model
# ============================================================

# 방법 1: state_dict 저장 (권장) / Method 1: Save state_dict (recommended)
# torch.save(lstm_model.state_dict(), 'lstm_model.pth')

# 방법 2: 전체 모델 저장 / Method 2: Save entire model
# torch.save(lstm_model, 'lstm_model_full.pth')

print("모델 저장 방법:")
print("# 저장 / Save")
print("torch.save(model.state_dict(), 'model.pth')")
print("")
print("# 로드 / Load")
print("model = LSTMModel(...)  # 동일한 구조로 생성")
print("model.load_state_dict(torch.load('model.pth'))")
print("model.eval()  # 평가 모드로 전환")

---
## 요약: 딥러닝 모델 선택 가이드 / Summary: Deep Learning Model Selection Guide

| 모델 | 적합한 데이터 | 특징 |
|------|---------------|------|
| **DNN** | 정형 데이터 (테이블) | 가장 기본, 빠름 |
| **CNN** | 이미지, 1D 시퀀스 | 지역적 패턴 학습 |
| **LSTM** | 시계열, 순차 데이터 | 장기 의존성 학습 |
| **GRU** | 시계열, 순차 데이터 | LSTM보다 빠름 |

### 핵심 체크리스트:
```python
# 1. 데이터 준비
X_train, X_test = train_test_split(X, y)
X_train_scaled = scaler.fit_transform(X_train)

# 2. DataLoader 생성
train_loader = DataLoader(dataset, batch_size=32, shuffle=True)

# 3. 모델 정의
model = LSTMModel(input_dim=1, hidden_dim=64).to(device)

# 4. 손실함수 & 옵티마이저
criterion = nn.MSELoss()  # 회귀
criterion = nn.CrossEntropyLoss()  # 분류
optimizer = optim.Adam(model.parameters(), lr=0.001)

# 5. 학습 루프
for epoch in range(epochs):
    for X_batch, y_batch in train_loader:
        optimizer.zero_grad()
        outputs = model(X_batch)
        loss = criterion(outputs, y_batch)
        loss.backward()
        optimizer.step()

# 6. 평가
model.eval()
with torch.no_grad():
    predictions = model(X_test)
```

In [ ]:
print("✅ 딥러닝 노트북 완료 / Deep Learning notebook complete!")
print("\n" + "="*60)
print("🎉 모든 치트시트 노트북 작성 완료!")
print("🎉 All cheatsheet notebooks completed!")
print("="*60)
print("\n파일 목록 / File List:")
print("  00_setup_and_preprocessing.ipynb - 환경 설정 및 전처리")
print("  01_regression_models.ipynb - 회귀 모델")
print("  02_classification_models.ipynb - 분류 모델")
print("  03_timeseries_arima.ipynb - 시계열 ARIMA")
print("  04_anomaly_detection.ipynb - 이상 탐지")
print("  05_deep_learning.ipynb - 딥러닝 (DNN/CNN/RNN)")